In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
import os
import random
import cv2
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D, Concatenate, concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.regularizers import l2
from keras import initializers
from keras.initializers import he_normal
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau, TensorBoard

from scipy.ndimage import gaussian_filter
from skimage import img_as_float
from skimage.morphology import reconstruction

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, log_loss

from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications.xception import Xception
from keras.applications.inception_v3 import InceptionV3
from keras.applications import inception_resnet_v2
from keras.applications.nasnet import NASNetLarge

Using TensorFlow backend.


In [2]:
def make_data(path, size=0):
    fileDataTrain = path

    inputTrainRaw = pd.read_json(fileDataTrain)

    inputTrainRaw.inc_angle.replace('na', 0, inplace=True)
    inputTrainRaw.inc_angle = inputTrainRaw.inc_angle.astype(float).fillna(0.0)

    inputTrain = inputTrainRaw[inputTrainRaw['inc_angle'] > 0]

    inputTrain['band_3'] = [(np.array(b1) + np.array(b2))/2 for b1, b2 in zip(inputTrain['band_1'], inputTrain['band_2'])]

    x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in inputTrain["band_1"]])
    x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in inputTrain["band_2"]])
    x_band3 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in inputTrain["band_3"]])

    if size:
        x_band1 = np.array([cv2.resize(img, (size, size), interpolation = cv2.INTER_CUBIC) for img in x_band1])
        x_band2 = np.array([cv2.resize(img, (size, size), interpolation = cv2.INTER_CUBIC) for img in x_band2])
        x_band3 = np.array([cv2.resize(img, (size, size), interpolation = cv2.INTER_CUBIC) for img in x_band3])
    

    x_data = np.concatenate([x_band1[:, :, :, np.newaxis], x_band2[:, :, :, np.newaxis], 
                             x_band3[:, :, :, np.newaxis]], axis=-1)

    del inputTrainRaw
    del inputTrain
    
    return np.array(x_data)

VGG16

In [3]:
vgg16 = VGG16(weights='imagenet', include_top=False, pooling='avg')
train = make_data('./test.json', size=224)
train_vgg16 = vgg16.predict(train, verbose=1)
vgg16 = None

8424/8424 [==============================] - 91s 11ms/step


In [4]:
with open('./encoded_test/encoded_x_vgg16_test.np', 'wb') as f:
    np.save(f, train_vgg16)

NASNet

In [3]:
nasnet = NASNetLarge(weights='imagenet', include_top=False, pooling='avg')
train = make_data('./test.json')

In [24]:
ind = int(0.25*len(train))
for i in range(0, 4):
    print (ind*i, ind*(i+1))
    train_part = train[ind*i:ind*(i+1)]
    train_part = np.array([cv2.resize(img, (331, 331), interpolation = cv2.INTER_CUBIC) for img in train_part])
    train_nasnet = nasnet.predict(train_part, verbose=1)
    with open('./encoded_test/encoded_x_nasnet_test_'+str(i)+'.np', 'wb') as f:
        np.save(f, train_nasnet)

0 1684
1684/1684 [==============================] - 115s 68ms/step
1684 3368
1684/1684 [==============================] - 115s 69ms/step
3368 5052
1684/1684 [==============================] - 116s 69ms/step
5052 6736
1684/1684 [==============================] - 115s 68ms/step
6736 8420
1684/1684 [==============================] - 115s 68ms/step


In [25]:
train_part = train[8420:]
train_part = np.array([cv2.resize(img, (331, 331), interpolation = cv2.INTER_CUBIC) for img in train_part])
train_nasnet = nasnet.predict(train_part, verbose=1)
with open('./encoded_test/encoded_x_nasnet_test_5.np', 'wb') as f:
    np.save(f, train_nasnet)

4/4 [==============================] - 0s 79ms/step


In [31]:
conc_out = np.load('./encoded_test/encoded_x_nasnet_test_0.np')
for i in range(1, 5):
    name = f'./encoded_test/encoded_x_nasnet_test_{i}.np'
    new_tab = np.load(name)
    conc_out = np.concatenate((conc_out, new_tab))

In [33]:
with open('./encoded_test/encoded_x_nasnet_test.np', 'wb') as f:
    np.save(f, conc_out)

VGG19

In [3]:
vgg19 = VGG19(weights='imagenet', include_top=False, pooling='avg')
train = make_data('./test.json', size=224)
train_vgg19 = vgg19.predict(train, verbose=1)
del vgg19
del train

8424/8424 [==============================] - 107s 13ms/step


In [4]:
with open('./encoded_test/encoded_x_vgg19_test.np', 'wb') as f:
    np.save(f, train_vgg19)

xception

In [34]:
xception = Xception(weights='imagenet', include_top=False, pooling='avg')
train = make_data('./test.json')
#train_xception = xception.predict(train, verbose=1)
ind = int(0.25*len(train))
for i in range(0, 4):
    print (ind*i, ind*(i+1))
    train_part = train[ind*i:ind*(i+1)]
    train_part = np.array([cv2.resize(img, (299, 299), interpolation = cv2.INTER_CUBIC) for img in train_part])
    train_nasnet = xception.predict(train_part, verbose=1)
    with open('./encoded_test/xception_parts/encoded_x_xception_test_'+str(i)+'.np', 'wb') as f:
        np.save(f, train_nasnet)

0 2106
2106/2106 [==============================] - 44s 21ms/step
2106 4212
2106/2106 [==============================] - 42s 20ms/step
4212 6318
2106/2106 [==============================] - 42s 20ms/step
6318 8424
2106/2106 [==============================] - 42s 20ms/step


In [35]:
conc_out = np.load('./encoded_test/xception_parts/encoded_x_xception_test_0.np')
for i in range(1, 4):
    name = f'./encoded_test/xception_parts/encoded_x_xception_test_{i}.np'
    new_tab = np.load(name)
    conc_out = np.concatenate((conc_out, new_tab))

In [36]:
conc_out.shape

(8424, 2048)

In [37]:
with open('./encoded_test/encoded_x_xception_test.np', 'wb') as f:
    np.save(f, conc_out)

inception_v3

In [38]:
inception = InceptionV3(weights='imagenet', include_top=False, pooling='avg')
train = make_data('./test.json')
#train_inception = inception.predict(train, verbose=1)
ind = int(0.25*len(train))
for i in range(0, 4):
    print (ind*i, ind*(i+1))
    train_part = train[ind*i:ind*(i+1)]
    train_part = np.array([cv2.resize(img, (299, 299), interpolation = cv2.INTER_CUBIC) for img in train_part])
    train_nasnet = inception.predict(train_part, verbose=1)
    with open('./encoded_test/inception_parts/encoded_x_inception_test_'+str(i)+'.np', 'wb') as f:
        np.save(f, train_nasnet)

0 2106
2106/2106 [==============================] - 27s 13ms/step
2106 4212
2106/2106 [==============================] - 24s 11ms/step
4212 6318
2106/2106 [==============================] - 24s 11ms/step
6318 8424
2106/2106 [==============================] - 24s 11ms/step


In [39]:
conc_out = np.load('./encoded_test/inception_parts/encoded_x_inception_test_0.np')
for i in range(1, 4):
    name = f'./encoded_test/inception_parts/encoded_x_inception_test_{i}.np'
    new_tab = np.load(name)
    conc_out = np.concatenate((conc_out, new_tab))

In [40]:
with open('./encoded_test/encoded_x_inception_test.np', 'wb') as f:
    np.save(f, conc_out)

InceptionResNetV2

In [41]:
InceptionResNetV2 = keras.applications.inception_resnet_v2.InceptionResNetV2(weights='imagenet', 
                                                                             include_top=False, pooling='avg')
train = make_data('./test.json')
#train_InceptionResNetV2 = InceptionResNetV2.predict(train, verbose=1)
ind = int(0.25*len(train))
for i in range(0, 4):
    print (ind*i, ind*(i+1))
    train_part = train[ind*i:ind*(i+1)]
    train_part = np.array([cv2.resize(img, (299, 299), interpolation = cv2.INTER_CUBIC) for img in train_part])
    train_nasnet = InceptionResNetV2.predict(train_part, verbose=1)
    with open('./encoded_test/InceptionResNetV2_parts/encoded_x_InceptionResNetV2_test_'+str(i)+'.np', 'wb') as f:
        np.save(f, train_nasnet)

0 2106
2106/2106 [==============================] - 47s 22ms/step
2106 4212
2106/2106 [==============================] - 44s 21ms/step
4212 6318
2106/2106 [==============================] - 45s 21ms/step
6318 8424
2106/2106 [==============================] - 44s 21ms/step


In [42]:
conc_out = np.load('./encoded_test/InceptionResNetV2_parts/encoded_x_InceptionResNetV2_test_0.np')
for i in range(1, 4):
    name = f'./encoded_test/InceptionResNetV2_parts/encoded_x_InceptionResNetV2_test_{i}.np'
    new_tab = np.load(name)
    conc_out = np.concatenate((conc_out, new_tab))

In [43]:
with open('./encoded_test/encoded_x_InceptionResNetV2_test.np', 'wb') as f:
    np.save(f, conc_out)

DenseNet201

In [7]:
DenseNet201 = keras.applications.densenet.DenseNet201(weights='imagenet', 
                                                      include_top=False, pooling='avg')
train = make_data('./test.json', size=224)
train_DenseNet201 = DenseNet201.predict(train, verbose=1)
DenseNet201 = None

8424/8424 [==============================] - 138s 16ms/step


In [8]:
with open('./encoded_test/encoded_x_DenseNet201_test.np', 'wb') as f:
    np.save(f, train_DenseNet201)

Save raw data

In [8]:
train = make_data('./train.json')

In [9]:
train = train[:,:,:,:2]

In [10]:
train.shape

(8424, 75, 75, 2)

In [11]:
train = train.reshape(8424, 75*75*2)

In [12]:
train.shape

(8424, 11250)

In [13]:
with open('./encoded_test/raw_test.np', 'wb') as f:
    np.save(f, train)